In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import glob, functools, tqdm, PIL
from multiprocess import Pool

def imread(fn):
    return np.array(PIL.Image.open(fn))

def imsize(fn):
    im = PIL.Image.open(fn)
    sz = im.size
    im.close()
    return sz

def is_landscape(fn):
    s = imsize(fn)
    return s[0] > s[1]

In [ ]:
train = pd.DataFrame({'path':glob.glob('../input/train/*/*')})
train['modelname'] = train.path.map(lambda p:p.split('/')[-2])

In [ ]:
def random_crop_fft(img, W):
    nr, nc = img.shape[:2]
    r1, c1 = np.random.randint(nr-W), np.random.randint(nc-W) 
    imgc = img[r1:r1+W, c1:c1+W, :]

    img1 = imgc - cv2.GaussianBlur(imgc, (3,3), 0)
    imgs1 = np.sum(img1, axis=2)
    
    sf = np.stack([
         np.fft.fftshift(np.fft.fft2( imgs1 )),
         np.fft.fftshift(np.fft.fft2( img1[:,:,0] - img1[:,:,1] )),
         np.fft.fftshift(np.fft.fft2( img1[:,:,1] - img1[:,:,2] )),
         np.fft.fftshift(np.fft.fft2( img1[:,:,2] - img1[:,:,0] )) ], axis=-1)
    return np.abs(sf)
    
def imread_residual_fft(fn, W, navg):
    #print(fn, rss())
    img = imread(fn).astype(np.float32) / 255.0
    return sum(map(lambda x:random_crop_fft(img, W), range(navg))) / navg



def noise_pattern(modelname, W, navg=256):
    files = train.path[train.modelname == modelname].values
    orientations = np.vectorize(is_landscape)(files)
    if np.sum(orientations) < len(orientations)//2:
        orientations = ~orientations
    files = files[orientations]

    from multiprocess import Pool
    with Pool() as pool:
        s = sum(tqdm.tqdm(pool.imap(lambda fn:imread_residual_fft(fn, W, navg), files), total=len(files), desc=modelname)) / len(files)
    
    return s

In [ ]:
def plot_model_features(modelname, W):
    s = noise_pattern(modelname, W)
    nchans = s.shape[2]
    nrows = (nchans + 3) // 4
    _, ax = plt.subplots(nrows, 4, figsize=(16, 4 * nrows))
    ax = ax.flatten()

    for c in range(nchans):
        eps = np.max(s[:,:,c]) * 1e-2
        s1 = np.log(s[:,:,c] + eps) - np.log(eps) 
        img = (s1 * 255 / np.max(s1)).astype(np.uint8)
        ax[c].imshow(cv2.equalizeHist(img))
        
    for ax1 in ax[nchans:]:
        ax1.axis('off')

    plt.show()
    
    
def plot_all_model_features(W):
    print("Feature Size={}".format(W))
    for modelname in train.modelname.unique():
        plot_model_features(modelname, W)
        
plot_all_model_features(W=128)



In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
        '../input/train/',  
        batch_size=1,
        class_mode='categorical')

In [ ]:
x, y = train_generator.next()
plt.imshow((x[0]*255).astype('uint8'));
print(list(train_generator.class_indices.keys())[np.argmax(y)])


In [ ]:
X_data, Y_data = [], []
for _ in tqdm(range(2750)):
    x, y = train_generator.next()
    X_data.append(x[0])
    Y_data.append(y[0])
X_data = np.asarray(X_data)
Y_data = np.asarray(Y_data)

In [ ]:
def get_model():
    input_img = Input((256, 256, 3))
    X = BatchNormalization()(input_img)
    X = Convolution2D(16, (3, 3), activation='relu')(X)
    X = BatchNormalization()(X)
    X = Convolution2D(16, (3, 3), activation='relu')(X)
    X = MaxPooling2D()(X)
    X = Convolution2D(32, (3, 3), activation='relu')(X)
    X = BatchNormalization()(X)
    X = Convolution2D(32, (3, 3), activation='relu')(X)
    X = GlobalMaxPooling2D()(X)
#     X = Flatten()(X)
    X = BatchNormalization()(X)
    X = Dense(512, activation='relu')(X)
    X = Dropout(0.2)(X)
    X = Dense(10, activation='softmax')(X)
    model = Model(inputs=input_img, outputs=X)

    model.compile(optimizer='adam', loss='categorical_crossentropy', 
                  metrics=['acc'])
    model.summary()

In [ ]:
model = get_model()


In [ ]:
model_history = model.fit(X_data, Y_data, batch_size=10, epochs=3, validation_split=0.2,
                          callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

In [ ]:
X_test = []
sub = pd.read_csv('../input/sample_submission.csv')

for fname in tqdm(sub['fname']):
    filepath = '../input/test/' + fname
    X_test.append(img_to_array(load_img(filepath, target_size=(256, 256))))
X_test = np.asarray(X_test)

In [ ]:
preds = model.predict(X_test, verbose=1)
preds = np.argmax(preds, axis=1)
preds = [list(train_generator.class_indices.keys())[p] for p in tqdm(preds)]

In [ ]:
sub['camera'] = preds
sub.to_csv('sub.csv', index=False)